In [23]:
%%writefile phonebook_search.cu
#include <bits/stdc++.h>
using namespace std;
#include <cuda.h>

struct Contact{
    char ind[65];
    char name[65];
    char phone_number[65];
};


string getInput(ifstream& file){
    string ans;
    char c;
    int readSuru = 0;
    while(file.get(c)){
        if(c == '\"'){
            if(readSuru == 1) break;
            readSuru = 1;
        }else{
            if(readSuru){
                ans.push_back(c);
            }
        }
    }
    return ans;
}

__device__ bool check(char* str1, char* str2){
    for(int i = 0; str1[i] != '\0'; i++){
        int flag = 1;
        for(int j = 0; str2[j] != '\0' ; j++){
            if(str1[i + j] != str2[j]){
                flag = 0;
                break;
            }
        }
        if(flag == 1) return true;
    }
    return false;
}


__global__ void myKernel(Contact* phoneBook, char* pat, int offset){
    int threadNumber = threadIdx.x + offset;
    if(check(phoneBook[threadNumber].name, pat)){
        printf("%s %s %s\n", phoneBook[threadNumber].ind, phoneBook[threadNumber].name, phoneBook[threadNumber].phone_number);
    }
}



int main(int argc, char* argv[])
{
    int threadLimit = atoi(argv[2]);

    ifstream myfile("/content/text_input");
    vector<Contact> phoneBook;


    while(myfile.peek() != EOF){

        string ind = getInput(myfile);
        string name = getInput(myfile);
        string phoneNum = getInput(myfile);

        Contact c;
        strcpy(c.ind, ind.c_str());
        strcpy(c.name, name.c_str());
        strcpy(c.phone_number, phoneNum.c_str());

        phoneBook.push_back(c);
    }

    string search_name = argv[1];
    char pat[65];
    strcpy(pat, search_name.c_str());


    char* d_pat;
    cudaMalloc(&d_pat, 65); //memory allocation
    cudaMemcpy(d_pat, pat, 65, cudaMemcpyHostToDevice); //copying to device

    int n = phoneBook.size();
    Contact* d_phoneBook;
    cudaMalloc(&d_phoneBook, n*sizeof(Contact));
    cudaMemcpy(d_phoneBook, phoneBook.data(), n * sizeof(Contact), cudaMemcpyHostToDevice);


    int bakiAche = n;
    int offset = 0;
    while(bakiAche > 0){
        int batchSize = min(threadLimit, bakiAche);
        myKernel<<<1,batchSize>>>(d_phoneBook, d_pat, offset);
        cudaDeviceSynchronize();

        bakiAche -= batchSize;
        offset += batchSize;
    }

}

Overwriting phonebook_search.cu


In [24]:
!nvcc -arch=sm_75 phonebook_search.cu -o asif

In [25]:
!time ./asif TANIA 1 > output.txt


real	0m0.130s
user	0m0.020s
sys	0m0.105s


In [10]:
!time ./asif TANIA 2 > output.txt


real	0m4.389s
user	0m4.128s
sys	0m0.235s


In [11]:
!time ./asif TANIA 3 > output.txt


real	0m3.866s
user	0m3.505s
sys	0m0.289s


In [12]:
!time ./asif TANIA 5 > output.txt


real	0m2.565s
user	0m2.342s
sys	0m0.216s


In [13]:
!time ./asif TANIA 10 > output.txt


real	0m1.939s
user	0m1.706s
sys	0m0.228s
